EX4

导入所需库

如缺少 gensim库则按以下步骤安装
根据gensim的版本不同（此处为3.6.0），初始化word2vec模型的时候，形参名称不同，，
最新版详细信息请参考
https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/word2vec.py
版本自查:
import gensim
gensim.__version__
     
安装3.6.0版本
pip uninstall gensim
pip install gensim==3.6.0
安装后重启环境


In [2]:
import os
import torch
import gensim
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from gensim.models import word2vec
student_id = 3218001578 # 填写学号
torch.cuda.is_available()

False

定义一些函数

In [2]:
def load_training_data(path='training_label.txt'):
    '''
    原始文件可以使用 notepad++软件预览
    读取训练数据
    training_label.txt:带label，需要读label
    training_nolabel.txt 则无需读label
    '''
    if 'training_label' in path:
        with open(path, 'r',encoding='utf-8') as f: 
            '''
            如出现：
            UnicodeDecodeError: 'gbk' codec can't decode byte 0xb9 in position 7958: illegal multibyte sequence
            去掉：
            encoding='utf-8'
            '''
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines] #  对每一行sentence移除换行符 '\n' 并以空白符' '所在位置分割单词
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines] # 每句 句首 是标签0或1
        return x, y
    else:
        with open(path, 'r',encoding='utf-8') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='testing_data'):
    # 把testing時需要的data讀進來
    with open(path, 'r',encoding='utf-8') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]   #移除换行符并以'，'所在位置分割，将前面的序号剔除
        X = [sen.split(' ') for sen in X]
    return X

# 验证集
def evaluation(outputs, labels):
    #outputs => probability (float)
    #labels => labels
    outputs[outputs>=0.5] = 1 # 大于等于0.5为有恶意
    outputs[outputs<0.5] = 0 
    correct = torch.sum(torch.eq(outputs, labels)).item() # 计数
    return correct

训练 word embedding 模块
该模块默认在 cpu 上训练，需要大概10分钟

生成以下三个文件
w2v_all.model
w2v_all.model.trainables.syn1neg.npy
w2v_all.model.wv.vectors.npy

使用时调用w2v_all.model即可

In [ ]:
import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec

def train_word2vec(sentences):
    # 训练word to vector 的 word embedding
    model = word2vec.Word2Vec(sentences, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)# 词向量维度=250，迭代10次
    return model

if __name__ == "__main__":
    print("loading training data ...")
    train_x, y = load_training_data('./data/training_label.txt')
    train_x_no_label = load_training_data('./data/training_nolabel.txt')

    print("loading testing data ...")
    test_x = load_testing_data('./data/testing_data.txt')

    model = train_word2vec(train_x + train_x_no_label + test_x)
    
    print("saving model ...")
    os.makedirs('./model',exist_ok=True)
    model.save(os.path.join('./model/w2v_all.model'))

定义数据处理函数

In [ ]:
from torch import nn
from gensim.models import Word2Vec

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path='./model/word2vec.model'):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
        
    def get_w2v_model(self):
        ''' 
        加载训练好的word2vec.model 
        并为其随机初始化一个 向量描述子
        '''
        # 把之前训练好的 word to vec 模型读进来
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
        
    def add_embedding(self, word):
        ''' 
        添加起始、终止符: <PAD>,<UNK>
        <PAD>,<UNK>的representation vector 由随机生成
        '''
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)
        
    def make_embedding(self, load=True):
        print("Get embedding ...")
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
            
        # 定义一个 word2idx 的 dictionary
        # 定义一个 idx2word 的 list
        # 定义一个 word2vector 的 list
        
        for i, word in enumerate(self.embedding.wv.vocab):
#             print('get words #{}'.format(i+1), end='\r')
            #e.g. self.word2index['he'] = 1 
            #e.g. self.index2word[1] = 'he'
            #e.g. self.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
#         print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 将 "<PAD>" 和 "<UNK>" 加进 embedding 里面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    
    def pad_sequence(self, sentence):
        '''统一句子长度'''
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    
    def sentence_word2idx(self):
        # 把句子里面的字转成相应的index
        sentence_list = []
        for i, sen in enumerate(self.sentences):
#             print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            # 将每个句子变成一样的长度
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        print("total sentences: {}".format(len(sentence_list)))
        return torch.LongTensor(sentence_list)
    
    def labels_to_tensor(self, y):
        # 把 labels 转成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)

In [ ]:
#定义dataset
import torch
from torch.utils import data

class TwitterDataset(data.Dataset):
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

LSTM Model

In [ ]:
import torch
from torch import nn
class LSTM_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(LSTM_Net, self).__init__()
        #  embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # 是否 embedding fix固定住，如果fix_embedding为False，在训练过程中，embedding也会跟着训练
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                                         nn.Linear(hidden_dim, 1),
                                         nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs, None)
        # x 的 dimension (batch, seq_len, hidden_size)
        # 取用 LSTM 最后一层的 hidden state
        x = x[:, -1, :] 
        x = self.classifier(x)
        return x

训练函数

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    
    model.train() 
    criterion = nn.BCELoss() #binary cross entropy loss
    t_batch = len(train) 
    v_batch = len(valid) 
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0

        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long) # device為"cuda"，將inputs轉成torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float) # device為"cuda"，將labels轉成torch.cuda.FloatTensor，因為等等要餵進criterion，所以型態要是float
            
            optimizer.zero_grad() # loss.backward()的gradient会累加，每训练一个epoch需要清零
            outputs = model(inputs) 
            outputs = outputs.squeeze() # 维度压缩，去掉dim为1的维度，使得outputs 和labels的维度匹配，能够进行criterion()
            loss = criterion(outputs, labels) 
            loss.backward() # 自动梯度
            optimizer.step() # 模型参数更新
            correct = evaluation(outputs, labels) # 计算此刻模型的training accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
            	epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size), end='\r')
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))

        # validation
        model.eval() # eval()模式，固定model的参数
        with torch.no_grad(): # 设置不计算梯度
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long) 
                labels = labels.to(device, dtype=torch.float) 
                
                outputs = model(inputs) 
                outputs = outputs.squeeze() 
                loss = criterion(outputs, labels) 
                correct = evaluation(outputs, labels) 
                
                total_acc += (correct / batch_size)
                total_loss += loss.item()

            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                # 如果validation的結果優於之前所有的結果，就把當下的模型存下來以備之後做預測時使用
                best_acc = total_acc
                #torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}/ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('-----------------------------------------------')
        model.train() # 将model的模式设置为train，这样optimizer就可以更新model的参数（因为前面设置了eval模式）

Test 函数

In [ ]:
# test.py
# 这个block用来对testing_data.txt做预测
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1 # 大于等於0.5為負面
            outputs[outputs<0.5] = 0 # 小于0.5為正面
            ret_output += outputs.int().tolist()
    
    return ret_output

Main

In [ ]:
# main.py
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
from sklearn.model_selection import train_test_split

# 通过torch.cuda.is_available()的返回值(True/False)判断GPU是否可用，True: device设为"cuda"，False为"cpu"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 数据、w2v模型路径初始化
data_dir = './data'
model_dir = './model' 
train_with_label = os.path.join(data_dir, 'training_label.txt')
train_no_label = os.path.join(data_dir, 'training_nolabel.txt')
testing_data = os.path.join(data_dir, 'testing_data.txt')

w2v_path = os.path.join(model_dir,'w2v_all.model')

'''
    训练参数初始化
    sen_len :  句子长度
    fix_embedding  :  是否在训练中固定embedding的大小
    batch_size :  批次大小
    epoch: 迭代次数
    learning_rate:  学习率    
'''

sen_len = 30 
fix_embedding = True # fix embedding during training
batch_size = 32
epoch = 5
lr = 0.001


print("loading data ...") # 读取'training_label.txt'和'training_nolabel.txt'
train_x, y = load_training_data(train_with_label)
train_x_no_label = load_training_data(train_no_label)
print(len(train_x),len(y),len(train_x_no_label))


print("Dealing data ...") # 对input 和 label 进行预处理
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)


# 定义一个model对象
model = LSTM_Net(embedding, embedding_dim=250, hidden_dim=250, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device) # device为"cuda"，model使用GPU來训练(喂进去的inputs的数据类型也需要是cuda tensor)

# 把data分為training data跟validation data(將一部份training data拿去當作validation data)
X_train, X_val, y_train, y_val = train_x[:90000], train_x[90000:], y[:90000], y[90000:]

# 把data做成dataset供dataloader取用
train_dataset = TwitterDataset(X=X_train, y=y_train)
val_dataset = TwitterDataset(X=X_val, y=y_val)

# 把data 转成 batch of tensors
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 0)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 0)

# 开始训练
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)

出现：
RuntimeError: DataLoader worker (pid(s) 17560, 24160, 12656, 15804, 19328, 23208, 22496, 20500) exited unexpectedly

更改num_worker=0

Predict and Write to csv file

In [ ]:
# 使用模型对测试数据做预测
data_dir = './data'
model_dir = './model' 
testing_data = os.path.join(data_dir, 'testing_data.txt')
w2v_path = os.path.join(model_dir,'w2v_all.model')

sen_len = 30
fix_embedding = True # fix embedding during training
batch_size = 128
epoch = 5
lr = 0.001

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print("loading testing data ...")
testing_data = os.path.join(data_dir, 'testing_data.txt')
test_x = load_testing_data(testing_data)
# print(len(test_x))
# print(test_x)
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 0)
print('\nload model ...')
# model = torch.load(os.path.join(model_dir, 'ckpt.model'))
model = torch.load('./model/ckpt.model')
outputs = testing(batch_size, test_loader, model, device)

tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
print("save csv ...")
os.makedirs('./submission',exist_ok=True)
tmp.to_csv(os.path.join('./submission/submission_%s.csv'%(student_id)), index=False)
print("Finish Predict")
